In [1]:
%load_ext autoreload 
%autoreload 2
import numpy as np
import random
from colossus.cosmology import cosmology
cosmology.setCosmology('planck18')
from colossus.halo import concentration, mass_defs
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from pylab import rcParams
from matplotlib import rc;rc('text', usetex=True);rc('font', weight='bold');matplotlib.rcParams['text.latex.preamble'] = r'\boldmath'
rcParams['font.family'] = 'serif'
rc('text.latex',preamble=r'\usepackage{/Volumes/Extreme_SSD/codes/master_thesis/code/configs/apjfonts}')
from matplotlib.lines import Line2D
sz_ft = 20
sz_lb = 14
color_arr = ['indigo', 'royalblue', 'lightseagreen', 'darkgreen', 'goldenrod', 'darkred']
from tqdm import tqdm
import cosmo
import lensing_estimator
from cosmo import CosmoCalc
import lensing
import foregrounds as fg
import experiments as exp
import sims
import stats
import tools
import ilc

## General Settings

In [2]:
nber_clus = 3000
nber_rand = 50000
nber_cov = 1000
nber_runs = 25
map_params = [180, 0.5, 180, 0.5] 
l, cl = CosmoCalc().cmb_power_spectrum()
z = 0.7
cluster = [2e14, cosmo.concentration_parameter(2e14, z, 0.674), z]
l, bl = exp.beam_power_spectrum(beam_fwhm = 1.4)
mass_int = np.arange(0, 4, 0.1)


l, cl_residual_so, res_ilc_dic_so, weights_arr_so = ilc.residuals_and_weights(components = 'all', experiment = 'so_lat', cov_from_sims = True)
cl_noise_so =  cl_residual_so

l, cl_residual_fyst, res_ilc_dic_fyst, weights_arr_fyst = ilc.residuals_and_weights(components = 'all', experiment = 'so_lat_fyst', cov_from_sims = True)
cl_noise_fyst =  cl_residual_fyst

l, cl_residual_s4wide, res_ilc_dic_s4wide, weights_arr_s4wide = ilc.residuals_and_weights(components = 'all', experiment = 's4wide', cov_from_sims = True)
cl_noise_s4wide =  cl_residual_s4wide

cl_residuals = [cl_residual_so, cl_residual_fyst, cl_residual_s4wide]

/Volumes/Extreme_SSD/codes/master_thesis/code/cosmo.py:98: RuntimeWarning: invalid value encountered in true_divide
  cl = dl * 2 * np.pi / (l*(l+1))
/Volumes/Extreme_SSD/codes/master_thesis/code/experiments.py:134: RuntimeWarning: divide by zero encountered in power
  nl_red = n_red*(l/elknee)**alphaknee
/Volumes/Extreme_SSD/codes/master_thesis/code/experiments.py:141: RuntimeWarning: divide by zero encountered in power
  nl_red2= n_red2*(l/elknee2)**alphaknee2
/Volumes/Extreme_SSD/codes/master_thesis/code/foregrounds.py:82: RuntimeWarning: divide by zero encountered in true_divide
  cl_rg = (2*np.pi)/(l * (l+1))*dl_rg
/Volumes/Extreme_SSD/codes/master_thesis/code/foregrounds.py:82: RuntimeWarning: invalid value encountered in multiply
  cl_rg = (2*np.pi)/(l * (l+1))*dl_rg
/Volumes/Extreme_SSD/codes/master_thesis/code/foregrounds.py:112: RuntimeWarning: divide by zero encountered in true_divide
  cl_cib =  (2*np.pi)/(l * (l+1)) * dl_cib
/Volumes/Extreme_SSD/codes/master_thesis/code/fo

## SNR for SO LAT, SO LAT + FYST, S4-Wide

In [3]:
maps_rand_so = sims.cmb_forecast_data(experiment = 'so_lat', map_params = map_params, l = l, cl = cl, cl_residual = cl_residual_so, bl = bl, nber_obs = nber_rand)
stack_bg_so = lensing_estimator.stack(map_params, maps_rand_so, l = l, cl = cl, cl_noise = cl_noise_so)                                       
del maps_rand_so
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_so.npy', stack_bg_so)  

maps_rand_fyst = sims.cmb_forecast_data(experiment = 'so_lat_fyst', map_params = map_params, l = l, cl = cl, cl_residual = cl_residual_fyst, bl = bl, nber_obs = nber_rand)
stack_bg_fyst = lensing_estimator.stack(map_params, maps_rand_fyst, l = l, cl = cl, cl_noise = cl_noise_fyst)                                       
del maps_rand_fyst
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_fyst.npy', stack_bg_fyst) 


maps_rand_s4wide = sims.cmb_forecast_data(experiment = 's4wide', map_params = map_params, l = l, cl = cl, cl_residual = cl_residual_s4wide, bl = bl, nber_obs = nber_rand)
stack_bg_s4wide = lensing_estimator.stack(map_params, maps_rand_s4wide, l = l, cl = cl, cl_noise = cl_noise_s4wide)                                       
del maps_rand_s4wide
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_s4wide.npy', stack_bg_s4wide)   

In [ ]:
covariance_matrix_so, _ = lensing_estimator.covariance_and_correlation_matrix(nber_cov*2, nber_clus, map_params, l, cl, bl = bl, experiment = 'so_lat', components = 'all', cl_noise = cl_noise_so)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_so.npy', covariance_matrix_so)

covariance_matrix_fyst, _ = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, bl = bl, experiment = 'so_lat_fyst', components = 'all', cl_noise = cl_noise_fyst)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_fyst.npy', covariance_matrix_fyst)

covariance_matrix_s4wide, _ = lensing_estimator.covariance_and_correlation_matrix(nber_cov, nber_clus, map_params, l, cl, bl = bl, experiment = 's4wide', components = 'all', cl_noise = cl_noise_s4wide)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_s4wide.npy', covariance_matrix_s4wide)

In [45]:
#model_profiles_so = lensing_estimator.model_profiles(nber_clus, map_params, l, cl, mass_int, z, bl = bl, cl_noise = cl_noise_so)
#np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_so.npy', model_profiles_so)

#model_profiles_fyst = lensing_estimator.model_profiles(nber_clus, map_params, l, cl, mass_int, z, bl = bl, cl_noise = cl_noise_fyst)
#np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_fyst.npy', model_profiles_fyst)

model_profiles_s4wide = lensing_estimator.model_profiles(nber_clus, map_params, l, cl, mass_int, z, bl = bl, cl_noise = cl_noise_s4wide)
np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_s4wide.npy', model_profiles_s4wide)

In [ ]:
stack_bg_so = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_so.npy')   
covariance_matrix_so = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_so.npy')
model_profiles_so = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_so.npy')
stack_bg_fyst = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_fyst.npy')   
covariance_matrix_fyst = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_fyst.npy')
model_profiles_fyst = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_fyst.npy')
stack_bg_s4wide = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/stack_bg_s4wide.npy')   
covariance_matrix_s4wide = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/covariance_matrix_s4wide.npy')
model_profiles_s4wide = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/model_profiles_s4wide.npy')


lk_arr_so = []  
lk_arr_fyst = []   
lk_arr_s4wide = []   
for i in tqdm(range(nber_runs)):  
    maps_clus = sims.cmb_forecast_test_data(map_params, l, cl, cluster = cluster, cl_residuals = cl_residuals, bl = bl, nber_obs = nber_clus)
    maps_clus_so, maps_clus_fyst, maps_clus_s4wide = maps_clus
    
    stack_clus_so = lensing_estimator.stack(map_params, maps_clus_so, l = l, cl = cl, cl_noise = cl_noise_so)
    stack_clus_fyst = lensing_estimator.stack(map_params, maps_clus_fyst, l = l, cl = cl, cl_noise = cl_noise_fyst)
    stack_clus_s4wide = lensing_estimator.stack(map_params, maps_clus_s4wide, l = l, cl = cl, cl_noise = cl_noise_s4wide)
        
    bins_so, lensing_dipole_profile_so, stack_dipole_so = lensing_estimator.lensing_dipole(map_params, stack_clus_so, stack_bg_so)
    bins_fyst, lensing_dipole_profile_fyst, stack_dipole_fyst = lensing_estimator.lensing_dipole(map_params, stack_clus_fyst, stack_bg_fyst)
    bins_s4wide, lensing_dipole_profile_s4wide, stack_dipole_s4wide = lensing_estimator.lensing_dipole(map_params, stack_clus_s4wide, stack_bg_s4wide)
    
    data_so = bins_so, lensing_dipole_profile_so, covariance_matrix_so
    data_fyst = bins_fyst, lensing_dipole_profile_fyst, covariance_matrix_fyst
    data_s4wide = bins_s4wide, lensing_dipole_profile_s4wide, covariance_matrix_s4wide
   
    
    likelihood_so, _, _ = stats.run_ml(data_so, model_profiles_so, mass_int)#, normalize = False)
    mass_int_lk_so, lk_so = likelihood_so
    lk_arr_so.append(lk_so)
    likelihood_fyst, _, _ = stats.run_ml(data_fyst, model_profiles_fyst, mass_int)#, normalize = False)
    mass_int_lk_fyst, lk_fyst = likelihood_fyst
    lk_arr_fyst.append(lk_fyst)
    likelihood_s4wide, _, _ = stats.run_ml(data_s4wide, model_profiles_s4wide, mass_int)#, normalize = False)
    mass_int_lk_s4wide, lk_s4wide = likelihood_s4wide
    lk_arr_s4wide.append(lk_s4wide)    
    
    del maps_clus
    del maps_clus_so
    del maps_clus_fyst
    del maps_clus_s4wide


    
comb_lk_so, mass_value_comb_lk_so, error_comb_lk_so = stats.combined_likelihood(mass_int_lk_so, lk_arr_so)#, normalize = False)
comb_lk_fyst, mass_value_comb_lk_fyst, error_comb_lk_fyst = stats.combined_likelihood(mass_int_lk_fyst, lk_arr_fyst)#, normalize = False) 
comb_lk_s4wide, mass_value_comb_lk_s4wide, error_comb_lk_s4wide = stats.combined_likelihood(mass_int_lk_s4wide, lk_arr_s4wide)#, normalize = False) 

snr_so = stats.signal_to_noise(comb_lk_so)
snr_fyst = stats.signal_to_noise(comb_lk_fyst)
snr_s4wide = stats.signal_to_noise(comb_lk_s4wide)

snr_arr = [snr_so, snr_fyst, snr_s4wide]

np.save('/Volumes/Extreme_SSD/codes/master_thesis/code/results/snr.npy', snr_arr) 

snr_results = "{0:11.3f}".format(snr_so) + "\n"
snr_results += "{0:11.3f}".format(snr_fyst) + "\n"          
snr_results += "{0:11.3f}".format(snr_s4wide) + "\n"

file = open('/Volumes/Extreme_SSD/codes/master_thesis/code/results/snr.txt',"w")
file.write(snr_results)
file.close() 

  0%|          | 0/25 [00:00<?, ?it/s]/Volumes/Extreme_SSD/anaconda/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:475: RuntimeWarning: Number of calls to function has reached maxfev = 1000.
  warnings.warn(errors[info][0], RuntimeWarning)
  4%|▍         | 1/25 [01:57<47:10, 117.95s/it]

In [ ]:
for i in range(25):
    plt.plot(mass_int_lk_so, lk_arr_so[i], color = 'red', alpha = 0.3)
plt.plot(mass_int_lk_so, comb_lk_so, color = 'black')
plt.subplots()
for i in range(25):
    plt.plot(mass_int_lk_fyst, lk_arr_fyst[i], color = 'red', alpha = 0.3)
plt.plot(mass_int_lk_fyst, comb_lk_fyst, color = 'black')
plt.subplots()
for i in range(25):
    plt.plot(mass_int_lk_s4wide, lk_arr_s4wide[i], color = 'red', alpha = 0.3)
plt.plot(mass_int_lk_s4wide, comb_lk_s4wide, color = 'black')

## Plots

In [ ]:
specs_dic, corr_noise_bands, rho = exp.specs('so_lat')
freq_arr = sorted( specs_dic.keys() )
beam_fwhm_arr, whitenoise_arr, rednoise_arr, elknee_arr, alphaknee_arr = [], [], [], [], []
for freq in freq_arr:
    beam_fwhm, noiseval_white, noiseval_red, elknee, alphaknee = specs_dic[freq]
    beam_fwhm_arr.append(beam_fwhm)
    whitenoise_arr.append(noiseval_white)
    rednoise_arr.append(noiseval_red)
    elknee_arr.append(elknee)
    alphaknee_arr.append(alphaknee) 


beam_arr = []
for i in range(len(beam_fwhm_arr)):
    l, bl = exp.beam_power_spectrum(beam_fwhm_arr[i])
    beam_arr.append(np.sqrt(bl))     
    
noise_power_white_arr = []
for i in range(len(whitenoise_arr)):
    l, nl = exp.white_noise_power_spectrum(whitenoise_arr[i])
    noise_power_white_arr.append(nl)    
    
noise_power_red_arr = []
for i in range(len(rednoise_arr)):
    l, nl = exp.red_noise_power_spectrum(rednoise_arr[i], elknee_arr[i], alphaknee_arr[i])
    noise_power_red_arr.append(nl)
    
l, nl_dic = exp.noise_power_spectra_dic(experiment = 'so_lat', deconvolve = False, use_cross_power = False)
noise_power_arr = []
for freq in freq_arr:
    noise_power_arr.append(nl_dic[freq])

In [ ]:
fig, axarr = plt.subplots(1,2, figsize=(13.5,5.5))
cmap = cm.jet
color_grad = [cmap(int(d)) for d in np.linspace(0, 255, len(freq_arr))]

for i in range(len(freq_arr)):
    axarr[0].plot(l, beam_arr[i],  color = color_grad[i], label = str(freq_arr[i])+' GHz')
axarr[0].set_xlabel(r'$\ell$', fontsize = sz_ft)
axarr[0].set_ylabel(r'$b_{\ell}$', fontsize = sz_ft)
axarr[0].set_ylim(0,1.3)
axarr[0].tick_params(labelsize = sz_lb)
axarr[0].legend(ncol = 3, prop={'size': sz_lb}) 

for i in range(len(freq_arr)):
    axarr[1].plot(l, noise_power_arr[i],  color = color_grad[i], ls = '-')
    axarr[1].plot(l, noise_power_white_arr[i],  color = color_grad[i], ls = '--')
    axarr[1].plot(l, noise_power_red_arr[i],  color = color_grad[i], ls = ':')

axarr[1].set_ylabel(r'$N_{\ell}\ [\mu$'+'K'+r'$^2]$', fontsize = sz_ft)
axarr[1].tick_params(labelsize = sz_lb)
axarr[1].set_xscale('log')
axarr[1].set_yscale('log')
axarr[1].set_xlim(100, 10000) 
axarr[1].set_ylim(1e-7, 1e4)
axarr[1].set_xlabel(r'$\ell$', fontsize = sz_ft)
patches = [Line2D([0], [0], color = c, marker='o', ls = '') for c in color_arr]
patches+= [Line2D([0], [0], color='black',linestyle='-')]
patches+= [Line2D([0], [0], color='black',linestyle='--')]
patches+= [Line2D([0], [0], color='black',linestyle=':')]
labels = [str(freq_arr[0])+' GHz', str(freq_arr[1])+' GHz', str(freq_arr[2])+' GHz', str(freq_arr[3])+' GHz',
          str(freq_arr[4])+' GHz', str(freq_arr[5])+' GHz','Total Noise', 'White Noise', 'Red Noise']
axarr[1].legend(patches, labels, ncol = 3, prop={'size': sz_lb})

fig.tight_layout(pad=3)
fig.savefig('/Volumes/Extreme_SSD/codes/master_thesis/code/figures/beam_and_noise_so_lat.eps', dpi = 200, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.set_yscale('log')
ax.plot(l, cl_residual_so-res_ilc_dic_so['ksz'], label = 'SO LAT', color = color_arr[0])
ax.plot(l, cl_residual_fyst-res_ilc_dic_fyst['ksz'], label = 'SO LAT + FYST', color = color_arr[1])
ax.plot(l, cl_residual_s4wide-res_ilc_dic_s4wide['ksz'], label = 'S4-Wide', color = color_arr[2])
ax.set_ylabel('Residual Power', fontsize = sz_ft)
ax.set_xlabel(r'$\ell$', fontsize = sz_ft)
ax.tick_params(labelsize = sz_lb)
ax.set_xlim(100, 10000)
ax.set_ylim(1e-6, 1e-2)
ax.legend(prop={'size': sz_lb}) 

plt.savefig('/Volumes/Extreme_SSD/codes/master_thesis/code/figures/residuals.eps', dpi = 200.)

In [ ]:
experiments = ['SO LAT', 'SO LAT + FYST', 'S4-Wide']
snr_arr = np.load('/Volumes/Extreme_SSD/codes/master_thesis/code/results/snr.npy') 
plt.plot(experiments, snr_arr, ls = '', marker = 'o')
plt.savefig('/Volumes/Extreme_SSD/codes/master_thesis/code/figures/forecasts.eps', dpi = 200., bbox_inches = 'tight', pad_inches = 0.1)